# Проект: Машинное обучение в бизнесе


## Описание проекта
Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.\
Шаги для выбора локации обычно такие:\
В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;\
Строят модель для предсказания объёма запасов в новых скважинах;\
Выбирают скважины с самыми высокими оценками значений;\
Определяют регион с максимальной суммарной прибылью отобранных скважин.\
Вам предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Постройте модель для определения региона, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap*.

## Условия задачи:
Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).\
При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.\
Бюджет на разработку скважин в регионе — 10 млрд рублей.\
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.\
После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.\
Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

## План проекта
1. [Загрузка и первичное изучение данных.](#start)
2. [Обучение и проверка модели для каждого региона](#train)\
    2.1 [Разбивка данных на обучающую и валидационную выборки в соотношении 75:25](#split)\
    2.2 [Обучение модели и получение предсказания на валидационной выборке](#traininig)\
    2.3 [Сохрание предсказания и правильных ответов на валидационной выборке](#save)\
    2.4 [Определение среднего запаса предсказанного сырья и RMSE модели](#mean)\
    2.5 [Анализ результатов](#analisys)
3. [Подготовка к расчету прибыли:](#prep_calc)\
    3.1 [Сохранение ключевых значений в отдельных переменных](#save_values)\
    3.2 [Расчет достаточного объёма сырья для безубыточной разработки новой скважины. Сравнение полученного объёма сырья со средним запасом в каждом регионе.](#value_calc)\
    3.3 [Вывод по этапу подготовки расчёта прибыли](#concl_3)
4. [Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели](#def_profit)\
    4.1 [Выбор скважин с максимальными значениями предсказаний.](#def_profit)\
    4.2 [Суммирование целевого значения объёма сырья, соответствующее этим предсказаниям.](#def_profit)\
    4.3 [Рассчет прибыли для полученного объёма сырья.](#def_profit)\
    4.4 [Вывод](#concl_4)
5. [Расчет рисков и прибыли для каждого региона:](#risks)\
    5.1 [Применение техники Bootstrap с 1000 выборок, чтобы найти распределение прибыли](#risks)\
    5.2 [Нахождение средней прибыли, 95%-й доверительного интервала и риска убытков](#risks)\
    5.3 [Вывод: предложение региона для разработки скважин с обоснованием](#concl_5)
6. [Общий вывод](#concl_6)
    
    
## Описание данных
Данные геологоразведки трёх регионов находятся в файлах:

/datasets/geo_data_0.csv. \
/datasets/geo_data_1.csv. \
/datasets/geo_data_2.csv. 

`id` — уникальный идентификатор скважины;\
`f0, f1, f2` — три признака точек (неважно, что они означают, но сами признаки значимы);\
`product` — объём запасов в скважине (тыс. баррелей).

### Загрузка и первичное изучение данных. <a id='start'></a>

Необходимые для исследования библиотеки будем хранить здесь.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

Импортируем данные для исследования

In [2]:
# функция пдля импорта данных
def import_df(data):
    try:
        df =  pd.read_csv('C:/Users/79153/Desktop/ya_projects/sprint_11/'+
                          str(data)+'.csv')
    except:
        df = pd.read_csv('/datasets/'+str(data)+'.csv')
    return df

# параметры импорта данных

set_names = ['geo_data_0','geo_data_1','geo_data_2']           # исследумые датасеты

# автоматизированный импорт данных в словарь c укороченным именем

sets = {}     

for i in set_names:
    sets[(i)[-1]]= import_df(i)

In [3]:
for i in range(3):
    print(f'Dataset № {i}')
    sets[str(i)].info()
    print('---'*15)
    print("")

Dataset № 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
---------------------------------------------

Dataset № 1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
---------------------------------------------

Dataset № 2
<c

In [4]:
for i in range(3):
    print(f'Dataset № {i}')
    print("")
    print(sets[str(i)].describe())
    print('---'*15)
    print("")

Dataset № 0

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347
---------------------------------------------

Dataset № 1

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%

In [5]:
for i in range(3):
    print(f'Dataset № {i}')
    print("")
    print(sets[str(i)].head(10))
    print('---'*15)
    print("")

Dataset № 0

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
5  wX4Hy  0.969570  0.489775 -0.735383   64.741541
6  tL6pL  0.645075  0.530656  1.780266   49.055285
7  BYPU6 -0.400648  0.808337 -5.624670   72.943292
8  j9Oui  0.643105 -0.551583  2.372141  113.356160
9  OLuZU  2.173381  0.563698  9.441852  127.910945
---------------------------------------------

Dataset № 1

      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305
5  HHckp  -3.327590  -2.205276  3.003647   84.03

In [6]:
for i in range(3):
    print(f'Dataset № {i}')
    print(f'Quntity of duplicates: {sets[str(i)].duplicated().sum()}')
    print('---'*15)
    print("")

Dataset № 0
Quntity of duplicates: 0
---------------------------------------------

Dataset № 1
Quntity of duplicates: 0
---------------------------------------------

Dataset № 2
Quntity of duplicates: 0
---------------------------------------------



### Вывод по первичному осмотру датасетов
Можно однозначно сказать:
1. Датасеты не содержат пропуски
2. Датасеты не содержат дубликаты

Судя по максимальным и минимальным значениям `f1`,`f2`,`f2` данные промасштабированы, кроме того адекватно оценить наличие выбросов невозможно так как параметры зашифрованы. Определить степень готовности данных к исследованию затруднительно, при условии, что данные синтетические, будем считать, что дополнительная предобработка данных в данном случае не нужна.

### 2. Обучение и проверка модели для каждого региона <a id='train'></a>

Автоматизируем разбиение и обучение модели с помощью функции, а так же сразу выведем интересующие нас метрики   <a id='split'><a id='save'><a id='training'><a id='mean'>

In [7]:
def splitter(df):
    # объявим признаки и цели
    feature_names = ['f0', 'f1', 'f2']
    target_name = ['product']
    
    feature = df[feature_names]
    target = df[target_name]
    
    # разбивка на выборки
    X_train, X_valid, y_train, y_valid = train_test_split(feature, target, test_size=0.25, random_state=123)
    
    # обучение модели
    model = LinearRegression()
    model.fit(X_train, y_train)
    predict = model.predict(X_valid)
    
    # подсчет метрик
    predict_mean = predict.mean()
    rmse = mean_squared_error(y_valid, predict)**0.5
    r2 = r2_score(y_valid,predict)
    
    # проверка разбивки
    print('Train shape:', X_train.shape)
    print('Valid shape:', X_valid.shape)
    
    # вывод метрик
    print('Mean facilities: {:.4f}'.format(predict_mean))
    print('RMSE модели: {:.4f}'.format(rmse))
    print('Сoefficient of determination (R2): {:.4f}'.format(r2))
        
    return y_valid, predict

In [8]:
valid = {}
predict = {}
for i in range(3):
    print(f'Dataset № {str(i)}')
    valid[str(i)], predict[str(i)] = splitter(sets[str(i)])
    print('---'*15)
    print("")

Dataset № 0
Train shape: (75000, 3)
Valid shape: (25000, 3)
Mean facilities: 92.5494
RMSE модели: 37.6479
Сoefficient of determination (R2): 0.2813
---------------------------------------------

Dataset № 1
Train shape: (75000, 3)
Valid shape: (25000, 3)
Mean facilities: 69.2800
RMSE модели: 0.8954
Сoefficient of determination (R2): 0.9996
---------------------------------------------

Dataset № 2
Train shape: (75000, 3)
Valid shape: (25000, 3)
Mean facilities: 95.0986
RMSE модели: 40.1280
Сoefficient of determination (R2): 0.1931
---------------------------------------------



### Вывод
1. Разбивка проведена верно
2. Наилучший показатель RMSE и R2 у модели обученной на датасете №1 
3. Лучшие средние производственные показатели у датасета №2, но как и у датасета №0 у него очень большое среднее отклонение и крайне низкий коэффициент детерминации

Вероятнее всего линейная регрессия не подходит для 2 и 0 датасетов, следует применить более сложную модель обучения, но условия тз не позволяют сделать этого.

### Подготовка к расчету прибыли <a id='prep_calc'></a>

Сохраним ключевые значения в отдельных переменных <a id='save_values'></a>
- `budget` - бюджет, рубль
- `wells` - выбранные скажины, шт.
- `research_wells` - исследуемые скважины, шт.
- `unit_profit` - доход с одного барелля сырья, рубль

In [9]:
budget = 10*10**9  
wells = 200
research_wells = 500
unit_profit = 450*10**3

Рассчитаем минимальнодостаточный объем для разработки и средний показатель по 200 лучшим скважинам в датасетах. 
Для расчета используем функцию. <a id='value_calc'></a>


In [10]:
min_value = budget/(wells*unit_profit)

def desire(data):
    best_200_mean = data['product'].sort_values(ascending=False)[:wells].mean()
    if best_200_mean > min_value:
        print('Средняя производительность 200 лучших скважин {:.2f}, что удовлетворяет условиям'.format(best_200_mean))
    else:
        print('Средняя производительность 200 лучших скважин  {:.2f}, что не удовлетворяет условиям'.format(best_200_mean))

In [11]:
print('Минимальный достаточный объем сырья для разработки: {:.2f}'.format(min_value))
print("")
for i in sets:
    print(f'Dataset №{i}')
    desire(sets[i])
    print("")

Минимальный достаточный объем сырья для разработки: 111.11

Dataset №0
Средняя производительность 200 лучших скважин 184.83, что удовлетворяет условиям

Dataset №1
Средняя производительность 200 лучших скважин 137.95, что удовлетворяет условиям

Dataset №2
Средняя производительность 200 лучших скважин 189.55, что удовлетворяет условиям



#### Вывод по этапу подготовки расчёта прибыли <a id='concl_3'></a>
Во всех трех датасетах производительность 200 лучших скважин удовлетворяет условиям. Последующая работа будет проводится на всех трех датасетах.

### Расчёт прибыли по выбранным скважинам и предсказаниям модели <a id='def_profit'></a>
Выбор скважин ведется исходя из предсказания, а подсчет по известному целевому признаку, для справки добавим так же вывод прибыли по непосредственно предсказанию.

In [12]:
def income(preds, target):
    target = pd.Series(target['product']).reset_index(drop=True)
    preds = pd.Series(*preds.reshape(1,-1))
    
    # берем из 500 рандомных 200 лучших
    sample_preds = preds.sample(n=research_wells, random_state=123)
    top_preds =  sample_preds.sort_values(ascending=False)[:wells]
    top_targets = target[top_preds.index]
    
    # подсчет искомых величин
    volume = sum(top_targets)
    income = volume * unit_profit - budget
    volume_pred = sum(top_preds)
    income_pred = volume_pred * unit_profit - budget
    
    # вывод показателей
    print('Суммарная целевая производительность: {:.2f}'.format(volume))
    print('Суммарная предсказанная производительность: {:.2f}'.format(volume_pred))
    print('Прибыль по целевому признаку: {:.2f} млн.р.'.format(income/(10**6)))
    print('Прибыль по предсказанию: {:.2f} млн.р.'.format(income_pred/(10**6)))

In [13]:
for i in range(3):
    print(f'Dataset № {i}')
    income(predict[str(i)], valid[str(i)])
    print('---'*15)
    print("")

Dataset № 0
Суммарная целевая производительность: 23078.71
Суммарная предсказанная производительность: 22655.10
Прибыль по целевому признаку: 385.42 млн.р.
Прибыль по предсказанию: 194.79 млн.р.
---------------------------------------------

Dataset № 1
Суммарная целевая производительность: 22882.63
Суммарная предсказанная производительность: 22883.35
Прибыль по целевому признаку: 297.18 млн.р.
Прибыль по предсказанию: 297.51 млн.р.
---------------------------------------------

Dataset № 2
Суммарная целевая производительность: 23024.03
Суммарная предсказанная производительность: 22810.42
Прибыль по целевому признаку: 360.81 млн.р.
Прибыль по предсказанию: 264.69 млн.р.
---------------------------------------------



### Вывод <a id='concl_4'></a>
В данном пункте мы расчитали прибыль по прогнозу и известному целевому признаку и убедились, что приведенные ранее метрики были правы - на датасетах №0 и №2 присутствует существенное отклонение по прибыли.

### Расчет рисков и прибыли для каждого региона <a id='risks'></a>
Подсчитаем риски и прибыли, применяя *boostrap*. Отбор скважин ведется по предсказанию, а расчет по известной целевой величине.

In [14]:
def risk(target, preds):
    
    # Заданные условиями параметры
    samples = 1000
    alpha = 0.05
    
    incomes = []
    
    target = pd.Series(target['product']).reset_index(drop=True)
    preds = pd.Series(*preds.reshape(1,-1))
    
    # вынос рандомайзера за пределы цикла
    rnd = np.random.RandomState(123)
    
    for _ in range(samples):
        
        # берем из 500 рандомных 200 лучших
        sample_preds = preds.sample(n=research_wells, replace=True, random_state=rnd)
        top_preds =  sample_preds.sort_values(ascending=False)[:wells]
        top_targets = target[top_preds.index]
    
        # подсчет производительности и прибыли
        volume = sum(top_targets)
        income = volume * unit_profit - budget
        # сохранение результата
        incomes.append(income)

    incomes = pd.Series(incomes)
    
    # подсчет искомых величин
    income_mean = incomes.mean()
    int_left = incomes.quantile(q = alpha*0.5)
    int_right = incomes.quantile(q = (1 - alpha*0.5))
    
    # подсчет доли отрицательных прибылей
    risk = (incomes < 0).mean()

    print('Средняя прибыль {:.2f} , млн. р.'.format(income_mean / 10**6))
    print('Доверительный интервал (95%): {:.2f} - {:.2f}, млн. р.'.format(int_left / 10**6, int_right / 10**6))
    print('Риск убытка: {:.2f} %'.format(risk * 100))

In [15]:
for i in range(3):
    print(f'Dataset № {i}')
    risk(valid[str(i)], predict[str(i)])
    print('---'*15)
    print("")

Dataset № 0
Средняя прибыль 477.42 , млн. р.
Доверительный интервал (95%): -57.99 - 974.82, млн. р.
Риск убытка: 4.10 %
---------------------------------------------

Dataset № 1
Средняя прибыль 479.19 , млн. р.
Доверительный интервал (95%): 58.73 - 874.42, млн. р.
Риск убытка: 0.90 %
---------------------------------------------

Dataset № 2
Средняя прибыль 343.45 , млн. р.
Доверительный интервал (95%): -231.38 - 860.84, млн. р.
Риск убытка: 9.90 %
---------------------------------------------



### Вывод: предложение региона для разработки скважин с обоснованием <a id='concl_5'></a>
Для разработки рекомендуется регион из датасета №1:
1. Данный регион обладает наименьшим риском убытков
2. Доверительный интервал прибыли начинается с положительного числа
3. Обладает наибольшей средней прибылью из трех регионов

### Общий вывод <a id='concl_6'></a>

В данной работе полноценная предобработка данных не была проведена, так как данные синтетические и, судя по значениям  предобработка уже была проведена заказчиком, однако на это нет прямого указания в ТЗ. Поэтому предобработка данных ограничилась поиском явных дубликатов и пропусков, так же проведен осмотр данных.

Проведено обучение модели *LinearRegression* для каждого региона по отдельности, получены предсказания производительности скважин. При этом выявлено, что обучение на датасетах прошло с различным успехом, наилучшие результаты показала модель обученная на датасете №2

В работе был рассчитан минимальнодостаточный объем сырья для безубыточной разработки новой скважины. С помощью функции рассчитана прибыль с каждого региона, а так же рассчитаны риски и доверительные интевалы для датасетов.

По итогу работы наиболее перспективным для разаботки является регион из датасета №1, поскольку:
1. Данный регион обладает наименьшим риском убытков
2. Доверительный интервал прибыли начинается с положительного числа
3. Обладает наибольшей средней прибылью из трех регионов

Вторым по перспективности регионом можно признать регион №0, так как при умеренном риске убытка 4.1% он предлагает практически такую же среднюю прибыль и доверительный интервал справа выше на 100 млн.р., то есть есть шанс заработать больше чем на регионе №1.

Регион №2 следует признать не перспективным ввиду высокого риска убытка.